In [9]:
import sqlite3
import pandas as pd

# Define database locations
db_a_path = r'C:\Users\NicholasRatti\OneDrive - Fernandina Capital, LLC\Fernandina Capital\Projects\Active\Python\Markov_Regime_Switching_Model\daily\output\financial_model.db'
db_b_path = r'C:\Users\NicholasRatti\OneDrive - Fernandina Capital, LLC\Fernandina Capital\Projects\Active\Python\Markov_Regime_Switching_Model\financial_model_test.db'
output_db_path = r'C:\Users\NicholasRatti\OneDrive - Fernandina Capital, LLC\Fernandina Capital\Projects\Active\Python\Markov_Regime_Switching_Model\comparison_output.db'

# Connect to the databases
conn_a = sqlite3.connect(db_a_path)
conn_b = sqlite3.connect(db_b_path)
output_conn = sqlite3.connect(output_db_path)

# Query to retrieve Date and Adjusted_Market_Regime values
query = "SELECT Date, Adjusted_Market_Regime FROM financial_data"  # Replace 'financial_data' with the actual table name

# Load data from both databases into DataFrames
df_a = pd.read_sql_query(query, conn_a)
df_b = pd.read_sql_query(query, conn_b)

# Close connections for input databases
conn_a.close()
conn_b.close()

# Set the 'Date' column as the index and ensure it's in datetime format
df_a['Date'] = pd.to_datetime(df_a['Date'])
df_b['Date'] = pd.to_datetime(df_b['Date'])
df_a.set_index('Date', inplace=True)
df_b.set_index('Date', inplace=True)

# Merge DataFrames on the Date index
merged_df = pd.merge(df_a, df_b, left_index=True, right_index=True, suffixes=('_A', '_B'))

# Compare values and create a 'Match' column
merged_df['Match'] = merged_df['Adjusted_Market_Regime_A'] == merged_df['Adjusted_Market_Regime_B']

# Filter only rows where Match is False
mismatched_df = merged_df[merged_df['Match'] == False]

# Rename columns for clarity
mismatched_df = mismatched_df.rename(columns={
    'Adjusted_Market_Regime_A': 'Value_From_Database_A',
    'Adjusted_Market_Regime_B': 'Value_From_Database_B'
})

# Write the mismatched rows to the output database
mismatched_df.to_sql('comparison_results', output_conn, if_exists='replace', index=True)

# Close output connection
output_conn.close()

print("Comparison of mismatched values complete. Results saved to the output database.")


Comparison of mismatched values complete. Results saved to the output database.
